# Trabajo práctico 7 - Contornos

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

Sobre la imagen de los bloques de la semana pasada encontrar los mismos y obtener los parámetros de área, perímetro y orientación. Adicionalmente indicar la relación de aspecto (largo Vs. ancho) de cada uno. Comparar los datos obtenidos obtenidos para cada bloque entre sí.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show()  

In [ ]:
#Cargamos la imagen a analizar
img = cv.imread('bloques1.jpg')
imgGray = cv.imread('bloques1.jpg', 0)
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
#img = cv.medianBlur(img,5) #Para eliminar los posibles ruidos
imgWidth = img.shape[1]
imgHeight = img.shape[0]

print("Img Width: {} \t Img Height: {}".format(imgWidth, imgHeight))
plotter(img, 'Original Image')
plotter(imgGray, 'Original Image Grayscale', grayScale=True)

In [ ]:
#Extraemos la parte que nos interesa analizar, o sea la tabla verde con los bloques
imgCut = img[800:3300, 200:2400, :]
imgCutGray = imgGray[800:3300, 200:2400]
plotter(imgCut, 'Image Cutted')
plotter(imgCutGray, 'Image Cutted Gray Scale', grayScale=True)

In [ ]:
#Ecualizamos el histograma de la imagen que cargamos en escala de grises
imgGrayMod = cv.equalizeHist(imgCutGray.copy())
plotter(imgGrayMod, 'Equalized', grayScale = True, step=100)

In [ ]:
#Aumentamos el brillo un 30%
imgGrayMod = np.clip(imgGrayMod + (255 * 0.30), 0, 255)
plotter(imgGrayMod, grayScale = True, step=100)

In [ ]:
imgPrueba = img.copy()
# Bloques numerados de arriba hacia abajo y de izq a derecha
#El tercer valor es el ancho del bloque para hacer la mascara
block1 = ((550, 980), (1350, 980), 345)
block2 = ((1500, 1600), (2050, 1600),125)
block3 = ((650, 2400), (1000, 2400), 750)
block4 = ((1300, 2400), (1600, 2400), 700)
block5 = ((1720, 2450), (1850, 2450),350)
block6 = ((1890, 2200), (2250, 2200),800)
blocks = [block1, block2, block3, block4, block5, block6]

# Marcamos una linea roja para ver correctamente sus posiciones
for block in blocks:
    cv.line(imgPrueba, block[0], block[1], (255,0,0), 5)

plotter(imgPrueba, step = 100)

In [ ]:
def createMask(img, samples, lowerMultiplier = 15, upperMultiplier = 6):
    meanColors = 0
    stdColors = 0
    for point1, point2, blockWidth in samples:
        colorMean, colorStd = cv.meanStdDev(img[point1[1]:point1[1] + blockWidth,point1[0]:point2[0], :])
        meanColors += colorMean
        stdColors += colorStd
    meanColors /= len(samples)
    stdColors /= len(samples)
    return cv.inRange(img, meanColors - stdColors * lowerMultiplier,  meanColors + stdColors * upperMultiplier)

In [ ]:
mask = createMask(img, blocks)
imgWithMask = cv.bitwise_and(img, img, mask=mask)
plotter(imgWithMask, 'Image With Mask' ,step = 100)

In [ ]:
imgWithMaskGray = cv.cvtColor(imgWithMask, cv.COLOR_RGB2GRAY)
imgWithMaskGray = imgWithMaskGray[800:3300, 200:2400]
plotter(imgWithMaskGray,'Gray Image With Mask', grayScale=True, step = 100)

In [ ]:
#Aplicamos un filtro de mediana a las dos imagenes para eliminar los ruidos
imgCutGray = cv.medianBlur(imgCutGray, 5)
imgWithMaskGray = cv.medianBlur(imgWithMaskGray, 5)
plotter(imgCutGray, 'Cut',grayScale=True)
plotter(imgWithMaskGray, 'Mask',grayScale=True)

In [ ]:
IMGS = [imgCutGray, imgWithMaskGray] #Estas son las imagenes que vamos a analizar
NAMES = ['Original', 'With Mask']

Binarizamos todas las imágenes con el algoritmo de Otsu, ya que las aplicaciones de apertura, cierre, erosión, etc se aplican sobre imágenes binarias. Antes de hacer esto primero graficamos los histogramas para analizar mejor las imágenes

In [ ]:
def plotHistograms(imgs, bins = 50):
    fig, axs = plt.subplots(len(imgs))
    fig.suptitle('Histograms', fontsize=18, fontweight='bold')
    for imgNum,img in enumerate(imgs):
        axs[imgNum].set_title(NAMES[imgNum], fontsize = 16, fontweight='bold')
        axs[imgNum].grid()
        axs[imgNum].hist(img.ravel(),bins,[0,256], color='orange')
    
    fig.set_size_inches(10, 8)
    fig.tight_layout(pad=3.0)

In [ ]:
plotHistograms([imgCutGray, imgWithMaskGray])

Observamos que los valores de threshold de 125 funcionará bien para ambas imágenes. Este valor lo ponemos como parámetro opcional en la siguiente función.

In [ ]:
def otsuBinarization(imgs, thresh = 127):
    result = []
    for imgNum, img in enumerate(imgs):
        ret, imgBin = cv.threshold(img,thresh,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
        plotter(imgBin,NAMES[imgNum], grayScale=True)
        result.append(imgBin)
    return result

In [ ]:
imgBin, imgMaskBin = otsuBinarization([imgCutGray, imgWithMaskGray])

Se pueden apreciar diferencias entre ambas binarizaciones. Si realizamos una binarización diretamente sobre la imagen original el bloque que se encuentra tapado se ve 'cortado', mientras que para la imagen con la máscara esto casi no sucede, ese bloque solo posee una pequeña mancha negra. Aplicaremos operaciones morfológicas a continuación para eliminar los ruidos y mejorar el problema que se tiene con el bloque tapado

In [ ]:
#PRUEBAS
imgTest = imgBin.copy()
plotter(imgTest, grayScale=True)
kernel = np.ones((100,100), np.uint8) #Influye bastante en la operacion morfolofica
imgTest = cv.morphologyEx(imgTest, cv.MORPH_OPEN, kernel)
kernelDil = np.ones((150,50), np.uint8)
plotter(cv.dilate(imgTest,kernelDil), grayScale=True)


In [ ]:
#PRUEBAS
imgTest2 = imgMaskBin.copy()
plotter(imgTest2, grayScale=True)
kernel = np.ones((125,125), np.uint8) #Influye bastante en la operacion morfolofica
plotter(cv.morphologyEx(imgTest2, cv.MORPH_OPEN, kernel), grayScale=True)

In [ ]:
kernel = np.ones((15,15), np.uint8)
plotter(cv.GaussianBlur(cv.dilate(imgTest,kernel),(5,5),10))

In [ ]:
kernel = np.ones((15,15), np.uint8)
contours, hier = cv.findContours(cv.dilate(imgTest,kernel), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
out = imgCut.copy()
cv.drawContours(out, contours, -1, (255,0,0),5)
plotter(out)

In [ ]:
out = imgCut.copy()
for contour in ctrs:
    approx = cv.approxPolyDP(contour, 0.06 * cv.arcLength(contour, True), True)
    cv.drawContours(out, [approx], 0, (0, 0, 0), 5)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
plotter(out)